# Fundamentals of MXNet Numpy Module

## Operator Namespaces for Imperative Programming
- `mxnet.numpy`: Regular NumPy operators
- `mxnet.numpy.random`: NumPy random operators
- `mxnet.numpy.linalg`: NumPy linear algebra operators
- `mxnet.numpy.ext`: Operators implemented in MXNet that do not exist in official NumPy

## Operator Namespaces for Gluon
`F` can be either `mxnet.ndarray` or `mxnet.symbol`.
- `F.np`: Regular NumPy operators
- `F.np.random`: NumPy random operators
- `F.np.linalg`: NumPy linear algebra operators
- `F.np.ext`: Operators implemented in MXNet that do not exist in official NumPy

## New `ndarray` and `symbol`
`mxnet.numpy.ndarray` and `mxnet.symbol.numpy._NumpySymbol` (not visible to users)
- Same name as in the official NumPy package
- Dispatch convience fluent method calls to MXNet Numpy operators
- Override many convenience fluent methods that do not exist in the official NumPy ndarray
- Make the behavior of built-in methods consistent with the official NumPy
    - Indexing: `__getitem__` and `__setitem__`
    - Many binary element-wise with broadcasting, not supported in `mxnet.symbol.Symbol`
    
## Examples of ndarray and symbol Basics
### Scalar and zero-size tensors

In [61]:
import mxnet as mx
from mxnet import numpy as np

# use numpy-compatible semantics
mx.set_np_compat(True)

# create a scalar tensor
x = np.array(3.14)
print(x)

3.14
<ndarray shape=() ctx=cpu(0)>


In [63]:
s = x.item()  # copy the element from the scalar tensor to a python scalar
print('s = {}'.format(str(s)))

s = 3.14


In [64]:
# create a scalar tensors with only one element 1.0
y = np.ones(())
print(y)

1.0
<ndarray shape=() ctx=cpu(0)>


In [65]:
# create a zero-size tensor
x = np.ones((5, 4, 0, 6))
print(x)

[]
<ndarray shape=(5, 4, 0, 6) ctx=cpu(0)>


In [66]:
# transpose the zero-size tensor
y = np.transpose(x)
print(y)

[]
<ndarray shape=(6, 0, 4, 5) ctx=cpu(0)>


### Conversion between classic and numpy ndarrays

In [67]:
# create a classic MXNet NDArray
x = mx.nd.random.uniform(shape=(2, 3))
print(x)


[[0.67568916 0.23107006 0.24488948]
 [0.5388685  0.21645726 0.773035  ]]
<NDArray 2x3 @cpu(0)>


In [68]:
# convert classic NDArray type to mxnet.numpy.ndarray with zero-copy
y = x.as_np_ndarray()
print(y)

[[0.67568916 0.23107006 0.24488948]
 [0.5388685  0.21645726 0.773035  ]]
<ndarray shape=(2, 3) ctx=cpu(0)>


In [69]:
# changing y's content changes x's content too
y[:] = 1
print(x)


[[1. 1. 1.]
 [1. 1. 1.]]
<NDArray 2x3 @cpu(0)>


In [70]:
# convert mxnet.numpy.ndarray to classic NDArray with zero-copy
z = y.as_classic_ndarray()
print(z)


[[1. 1. 1.]
 [1. 1. 1.]]
<NDArray 2x3 @cpu(0)>


In [71]:
# changing z's content changes y's content too
z[:] = 2
print(y)

[[2. 2. 2.]
 [2. 2. 2.]]
<ndarray shape=(2, 3) ctx=cpu(0)>


### Binary element-wise with broadcasting in new and old symbols

In [72]:
from mxnet import gluon
class TestBinaryBroadcast(gluon.HybridBlock):
    def hybrid_forward(self, F, x1, x2):
        print("x1 type:", str(type(x1)))
        print("x2 type:", str(type(x2)))
        return x1 + x2

net = TestBinaryBroadcast()
x1 = mx.nd.ones((2, 1))
x2 = mx.nd.ones((1, 3))
out = net(x1, x2)  # ok: imperative execution supports broadcasting
print(out)

x1 type: <class 'mxnet.ndarray.ndarray.NDArray'>
x2 type: <class 'mxnet.ndarray.ndarray.NDArray'>

[[2. 2. 2.]
 [2. 2. 2.]]
<NDArray 2x3 @cpu(0)>


In [73]:
net.hybridize()  # mark the block for execution using a computational graph
try:
    out = net(x1, x2)  # error: old symbol `+` operation does not support broadcasting
    assert False  # should not reach here
except mx.MXNetError:
    print("ERROR: cannot perform broadcast add for two symbols of mxnet.sym.Symbol")

x1 type: <class 'mxnet.symbol.symbol.Symbol'>
x2 type: <class 'mxnet.symbol.symbol.Symbol'>
ERROR: cannot perform broadcast add for two symbols of mxnet.sym.Symbol


In [58]:
net = TestBinaryBroadcast()  # Create a new block object to clear the graph
net.hybridize()  # mark the block for execution using a computational graph

x1 = x1.as_np_ndarray()  # convert x1 to np.ndarray so that _NumpySymbol will be used in graph construction
x2 = x2.as_np_ndarray()  # convert x2 to np.ndarray so that _NumpySymbol will be used in graph construction
out = net(x1, x2)  # ok: `+` operation supports broadcasting for _NumpySymbol
print(out)  # mxnet.numpy.ndarray type, because it's from a np operator

x1 type: <class 'mxnet.symbol.numpy._symbol._NumpySymbol'>
x2 type: <class 'mxnet.symbol.numpy._symbol._NumpySymbol'>
[[2. 2. 2.]
 [2. 2. 2.]]
<ndarray shape=(2, 3) ctx=cpu(0)>


## A Simple Linear Regression Model
Let's consider a simple linear regression model as the following.
Given dataset `{x, y}`, where `x`s represent input examples and `y`s represent observed data, find the parameters `w1` and `w2` for the following model.
```
y_pred = np.dot(np.maximum(np.dot(x, w1), 0), w2)
```

## MXNet Numpy Operators in Imperative Programming

In [74]:
import mxnet as mx
from mxnet import numpy as np
from mxnet import autograd
try:
    from mxboard import SummaryWriter
except ImportError:
    SummaryWriter = None

# create a summary writer for visualization
sw = SummaryWriter(logdir='./logs', flush_secs=2) if SummaryWriter is not None else None

# Use numpy-compatible semantics to support scalar tensors
mx.set_np_compat(True)

# N is number of examples; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = mx.nd.random.normal(shape=(N, D_in)).as_np_ndarray()  # x is of type mxnet.numpy.ndarray
y = mx.nd.random.normal(shape=(N, D_out)).as_np_ndarray()  # y is of type mxnet.numpy.ndarray

# Randomly initialize weights
w1 = mx.nd.random.normal(shape=(D_in, H)).as_np_ndarray()  # w1 is of type mxnet.numpy.ndarray
w1.attach_grad()  # w1.grad is of type mxnet.numpy.ndarray
w2 = mx.nd.random.normal(shape=(H, D_out)).as_np_ndarray()  # w2 is of type mxnet.numpy.ndarray
w2.attach_grad()  # w2.grad is of type mxnet.numpy.ndarray

learning_rate = 1e-6


for t in range(1000):
    with autograd.record():
        # Forward pass: compute predicted y
        h = x.dot(w1)  # equivalent to np.dot(x, w1)
        h_relu = np.ext.relu(h)  # equivalent to mx.nd.relu(h)
        y_pred = h_relu.dot(w2)  # equivalent to np.dot(h_relu, w2)

        # Compute loss
        # (y_pred - y) ** 2 calls np.ndarray.__pow__
        # sum() calls np.sum() which should return a scalar tensor
        loss = ((y_pred - y) ** 2).sum()
    # Note that the print function will invoke loss.asnumpy()
    print(t, loss)  # loss is a scalar tensor of type mxnet.numpy.ndarray
    loss.backward()

    # Update weights
    w1 -= learning_rate * w1.grad
    w2 -= learning_rate * w2.grad

    if sw is not None:
        sw.add_scalar('loss', loss.item(), global_step=t)  # loss.item() copies the tensor element to a python scalar
        if t % 50 == 0:
            sw.add_histogram(tag='w1', values=w1, global_step=t)
            sw.add_histogram(tag='w2', values=w2, global_step=t)

if sw is not None:
    sw.close()

0 30583507.8675299
<ndarray shape=() ctx=cpu(0)>
1 28463753.093591757
<ndarray shape=() ctx=cpu(0)>
2 30397192.154316917
<ndarray shape=() ctx=cpu(0)>
3 31001065.701931268
<ndarray shape=() ctx=cpu(0)>
4 27362428.668684393
<ndarray shape=() ctx=cpu(0)>
5 19413441.715605147
<ndarray shape=() ctx=cpu(0)>
6 11474426.529398067
<ndarray shape=() ctx=cpu(0)>
7 6019864.391239911
<ndarray shape=() ctx=cpu(0)>
8 3171177.319677336
<ndarray shape=() ctx=cpu(0)>
9 1818891.90549522
<ndarray shape=() ctx=cpu(0)>
10 1181206.3533717224
<ndarray shape=() ctx=cpu(0)>
11 855013.7450783919
<ndarray shape=() ctx=cpu(0)>
12 667014.9513980141
<ndarray shape=() ctx=cpu(0)>
13 544032.5234220314
<ndarray shape=() ctx=cpu(0)>
14 455366.5327710896
<ndarray shape=() ctx=cpu(0)>
15 386966.8337304372
<ndarray shape=() ctx=cpu(0)>
16 332209.28685336973
<ndarray shape=() ctx=cpu(0)>
17 287172.5239778813
<ndarray shape=() ctx=cpu(0)>
18 249598.0872202803
<ndarray shape=() ctx=cpu(0)>
19 217963.54169952107
<ndarray shap

205 2.0024782853880243
<ndarray shape=() ctx=cpu(0)>
206 1.9101641705726298
<ndarray shape=() ctx=cpu(0)>
207 1.8219825997771262
<ndarray shape=() ctx=cpu(0)>
208 1.7379100467427213
<ndarray shape=() ctx=cpu(0)>
209 1.6580087253671856
<ndarray shape=() ctx=cpu(0)>
210 1.5816025963892706
<ndarray shape=() ctx=cpu(0)>
211 1.508973034414094
<ndarray shape=() ctx=cpu(0)>
212 1.4393580410692781
<ndarray shape=() ctx=cpu(0)>
213 1.3733472652666263
<ndarray shape=() ctx=cpu(0)>
214 1.3102693440962527
<ndarray shape=() ctx=cpu(0)>
215 1.2500349649579103
<ndarray shape=() ctx=cpu(0)>
216 1.1927516726519982
<ndarray shape=() ctx=cpu(0)>
217 1.137938256649908
<ndarray shape=() ctx=cpu(0)>
218 1.085901497863119
<ndarray shape=() ctx=cpu(0)>
219 1.0360603797721968
<ndarray shape=() ctx=cpu(0)>
220 0.9886968248675567
<ndarray shape=() ctx=cpu(0)>
221 0.9435318537216792
<ndarray shape=() ctx=cpu(0)>
222 0.9002208665761496
<ndarray shape=() ctx=cpu(0)>
223 0.859025937446395
<ndarray shape=() ctx=cpu(0

418 0.0003182329937012618
<ndarray shape=() ctx=cpu(0)>
419 0.0003100011944885406
<ndarray shape=() ctx=cpu(0)>
420 0.0003027049226713707
<ndarray shape=() ctx=cpu(0)>
421 0.0002951453962293238
<ndarray shape=() ctx=cpu(0)>
422 0.0002872323396980875
<ndarray shape=() ctx=cpu(0)>
423 0.00027966232621109977
<ndarray shape=() ctx=cpu(0)>
424 0.0002725791607544187
<ndarray shape=() ctx=cpu(0)>
425 0.0002666564350377182
<ndarray shape=() ctx=cpu(0)>
426 0.0002603579371097814
<ndarray shape=() ctx=cpu(0)>
427 0.0002543142936318882
<ndarray shape=() ctx=cpu(0)>
428 0.00024914397795701426
<ndarray shape=() ctx=cpu(0)>
429 0.0002437161821449707
<ndarray shape=() ctx=cpu(0)>
430 0.00023855837631731047
<ndarray shape=() ctx=cpu(0)>
431 0.0002328217582505121
<ndarray shape=() ctx=cpu(0)>
432 0.00022722907680339244
<ndarray shape=() ctx=cpu(0)>
433 0.0002222060689299199
<ndarray shape=() ctx=cpu(0)>
434 0.00021712411611141125
<ndarray shape=() ctx=cpu(0)>
435 0.00021192676582065806
<ndarray shape=(

590 2.4276546342252847e-05
<ndarray shape=() ctx=cpu(0)>
591 2.4050617317819545e-05
<ndarray shape=() ctx=cpu(0)>
592 2.3692909408748464e-05
<ndarray shape=() ctx=cpu(0)>
593 2.3643774534357065e-05
<ndarray shape=() ctx=cpu(0)>
594 2.3478738471680027e-05
<ndarray shape=() ctx=cpu(0)>
595 2.3223246858800062e-05
<ndarray shape=() ctx=cpu(0)>
596 2.3094498732195712e-05
<ndarray shape=() ctx=cpu(0)>
597 2.2911456816404607e-05
<ndarray shape=() ctx=cpu(0)>
598 2.281473177449811e-05
<ndarray shape=() ctx=cpu(0)>
599 2.2665588339784715e-05
<ndarray shape=() ctx=cpu(0)>
600 2.2540706841683855e-05
<ndarray shape=() ctx=cpu(0)>
601 2.2397249842598388e-05
<ndarray shape=() ctx=cpu(0)>
602 2.2196343611710634e-05
<ndarray shape=() ctx=cpu(0)>
603 2.2068208186773464e-05
<ndarray shape=() ctx=cpu(0)>
604 2.200789568494732e-05
<ndarray shape=() ctx=cpu(0)>
605 2.1662697970853406e-05
<ndarray shape=() ctx=cpu(0)>
606 2.1670226707709572e-05
<ndarray shape=() ctx=cpu(0)>
607 2.1495529849840933e-05
<ndarr

740 1.0145177559222132e-05
<ndarray shape=() ctx=cpu(0)>
741 1.0054893305217784e-05
<ndarray shape=() ctx=cpu(0)>
742 1.0039937382828358e-05
<ndarray shape=() ctx=cpu(0)>
743 9.981997802403254e-06
<ndarray shape=() ctx=cpu(0)>
744 9.955391765461077e-06
<ndarray shape=() ctx=cpu(0)>
745 9.92333156156315e-06
<ndarray shape=() ctx=cpu(0)>
746 9.838239903981957e-06
<ndarray shape=() ctx=cpu(0)>
747 9.82860525570832e-06
<ndarray shape=() ctx=cpu(0)>
748 9.785722703088348e-06
<ndarray shape=() ctx=cpu(0)>
749 9.736639513339626e-06
<ndarray shape=() ctx=cpu(0)>
750 9.698717650133588e-06
<ndarray shape=() ctx=cpu(0)>
751 9.679464560677538e-06
<ndarray shape=() ctx=cpu(0)>
752 9.644895746199929e-06
<ndarray shape=() ctx=cpu(0)>
753 9.583351699230143e-06
<ndarray shape=() ctx=cpu(0)>
754 9.569756327832031e-06
<ndarray shape=() ctx=cpu(0)>
755 9.544720377685674e-06
<ndarray shape=() ctx=cpu(0)>
756 9.488677214783588e-06
<ndarray shape=() ctx=cpu(0)>
757 9.427434050440775e-06
<ndarray shape=() ctx

951 5.131926865045017e-06
<ndarray shape=() ctx=cpu(0)>
952 5.134807113935103e-06
<ndarray shape=() ctx=cpu(0)>
953 5.1036034350721016e-06
<ndarray shape=() ctx=cpu(0)>
954 5.108844745471686e-06
<ndarray shape=() ctx=cpu(0)>
955 5.094602056124331e-06
<ndarray shape=() ctx=cpu(0)>
956 5.075664423329074e-06
<ndarray shape=() ctx=cpu(0)>
957 5.05209753723021e-06
<ndarray shape=() ctx=cpu(0)>
958 5.0431675646910035e-06
<ndarray shape=() ctx=cpu(0)>
959 5.012941636208662e-06
<ndarray shape=() ctx=cpu(0)>
960 4.999564939722062e-06
<ndarray shape=() ctx=cpu(0)>
961 5.0270430448813574e-06
<ndarray shape=() ctx=cpu(0)>
962 5.014227540501431e-06
<ndarray shape=() ctx=cpu(0)>
963 5.000636092659244e-06
<ndarray shape=() ctx=cpu(0)>
964 4.98694181192208e-06
<ndarray shape=() ctx=cpu(0)>
965 4.979996630325584e-06
<ndarray shape=() ctx=cpu(0)>
966 4.951172870557713e-06
<ndarray shape=() ctx=cpu(0)>
967 4.935392461356131e-06
<ndarray shape=() ctx=cpu(0)>
968 4.930455768745873e-06
<ndarray shape=() ctx

## MXNet Numpy Operators in Gluon `HybridBlock`

In [75]:
import mxnet as mx
from mxnet import gluon, autograd
try:
    from mxboard import SummaryWriter
except ImportError:
    SummaryWriter = None

# create a summary writer for visualization
sw = SummaryWriter(logdir='./logs', flush_secs=2) if SummaryWriter is not None else None

# Use numpy-compatible semantics to support scalar tensors
mx.set_np_compat(True)


class LinearRegression(gluon.HybridBlock):
    def __init__(self, num_input_dim=1000, num_hidden_dim=100, num_output_dim=10):
        super(LinearRegression, self).__init__()
        with self.name_scope():
            self.w1 = self.params.get('w1', shape=(num_input_dim, num_hidden_dim),
                                      allow_deferred_init=True)
            self.w2 = self.params.get('w2', shape=(num_hidden_dim, num_output_dim),
                                      allow_deferred_init=True)

    def hybrid_forward(self, F, x, w1, w2):
        h = x.dot(w1)  # equivalent to F.np.dot(x, w1)
        h_relu = F.np.ext.relu(h)  # equivalent to F.relu(h)
        y_pred = h_relu.dot(w2)  # equivalent to F.np.dot(h_relu, w2)
        return y_pred


class TotalLoss(gluon.HybridBlock):
    def hybrid_forward(self, F, pred, label):
        return ((pred - label) ** 2).sum()  # equivalent to F.np.sum(F.np.square(pred - label))


regressor = LinearRegression()
regressor.initialize(mx.init.Normal())
regressor.hybridize()

# Create random input and output data
x = mx.nd.random.normal(shape=(64, 1000)).as_np_ndarray()  # x is of type mxnet.numpy.ndarray
y = mx.nd.random.normal(shape=(64, 10)).as_np_ndarray()  # y is of type mxnet.numpy.ndarray

total_loss = TotalLoss()
trainer = gluon.Trainer(regressor.collect_params(), 'sgd', {'learning_rate': 1e-3, 'momentum': 0.9})

for t in range(1000):
    with autograd.record():
        output = regressor(x)  # output is a type of np.ndarray because np.dot is the last op in the network
        loss = total_loss(output, y)  # loss is a scalar np.ndarray
    loss.backward()
    print(t, loss)  # note that loss.asnumpy() is called
    trainer.step(1)
    if sw is not None:
        sw.add_scalar('loss', loss.item(), global_step=t)  # loss.item() copies the tensor element to a python scalar
        if t % 50 == 0:
            for k, v in regressor.collect_params().items():
                sw.add_histogram(tag=k, values=v.data(), global_step=t)

if sw is not None:
    sw.close()

0 579.4144268617497
<ndarray shape=() ctx=cpu(0)>
1 556.4602369686031
<ndarray shape=() ctx=cpu(0)>
2 513.0694522222475
<ndarray shape=() ctx=cpu(0)>
3 429.9187940477759
<ndarray shape=() ctx=cpu(0)>
4 294.52651197032446
<ndarray shape=() ctx=cpu(0)>
5 144.37867825678688
<ndarray shape=() ctx=cpu(0)>
6 180.7942728908102
<ndarray shape=() ctx=cpu(0)>
7 163.45929258736442
<ndarray shape=() ctx=cpu(0)>
8 121.02842934141098
<ndarray shape=() ctx=cpu(0)>
9 101.11230996315157
<ndarray shape=() ctx=cpu(0)>
10 128.31593823389687
<ndarray shape=() ctx=cpu(0)>
11 100.1871811835118
<ndarray shape=() ctx=cpu(0)>
12 64.87949137351052
<ndarray shape=() ctx=cpu(0)>
13 66.41600744650378
<ndarray shape=() ctx=cpu(0)>
14 48.992466748077476
<ndarray shape=() ctx=cpu(0)>
15 46.041289013454666
<ndarray shape=() ctx=cpu(0)>
16 38.849225931447286
<ndarray shape=() ctx=cpu(0)>
17 40.07425207329399
<ndarray shape=() ctx=cpu(0)>
18 32.45095257962328
<ndarray shape=() ctx=cpu(0)>
19 29.924090651959318
<ndarray s

180 8.815068029896511e-07
<ndarray shape=() ctx=cpu(0)>
181 8.900138068401384e-07
<ndarray shape=() ctx=cpu(0)>
182 7.709548885239184e-07
<ndarray shape=() ctx=cpu(0)>
183 7.450055287318985e-07
<ndarray shape=() ctx=cpu(0)>
184 5.873908921222998e-07
<ndarray shape=() ctx=cpu(0)>
185 5.744630319874655e-07
<ndarray shape=() ctx=cpu(0)>
186 5.009225882801442e-07
<ndarray shape=() ctx=cpu(0)>
187 4.78328282593754e-07
<ndarray shape=() ctx=cpu(0)>
188 3.8269554183999535e-07
<ndarray shape=() ctx=cpu(0)>
189 3.71885413642456e-07
<ndarray shape=() ctx=cpu(0)>
190 3.275736110671182e-07
<ndarray shape=() ctx=cpu(0)>
191 3.139449568867822e-07
<ndarray shape=() ctx=cpu(0)>
192 2.540541410335495e-07
<ndarray shape=() ctx=cpu(0)>
193 2.4870893984303005e-07
<ndarray shape=() ctx=cpu(0)>
194 2.135481120305819e-07
<ndarray shape=() ctx=cpu(0)>
195 2.0506699530565414e-07
<ndarray shape=() ctx=cpu(0)>
196 1.6699345941478816e-07
<ndarray shape=() ctx=cpu(0)>
197 1.672973993843349e-07
<ndarray shape=() ct

366 9.583991698267633e-12
<ndarray shape=() ctx=cpu(0)>
367 9.864812840974487e-12
<ndarray shape=() ctx=cpu(0)>
368 1.0364411074309044e-11
<ndarray shape=() ctx=cpu(0)>
369 8.630546449083171e-12
<ndarray shape=() ctx=cpu(0)>
370 9.58487077955348e-12
<ndarray shape=() ctx=cpu(0)>
371 9.627293269363771e-12
<ndarray shape=() ctx=cpu(0)>
372 8.822139768932882e-12
<ndarray shape=() ctx=cpu(0)>
373 9.758966275737928e-12
<ndarray shape=() ctx=cpu(0)>
374 9.565188078446488e-12
<ndarray shape=() ctx=cpu(0)>
375 8.387528115761734e-12
<ndarray shape=() ctx=cpu(0)>
376 9.31537330183408e-12
<ndarray shape=() ctx=cpu(0)>
377 9.934673987329126e-12
<ndarray shape=() ctx=cpu(0)>
378 8.700355540181004e-12
<ndarray shape=() ctx=cpu(0)>
379 8.237079109616775e-12
<ndarray shape=() ctx=cpu(0)>
380 9.793622035851775e-12
<ndarray shape=() ctx=cpu(0)>
381 9.976530604920543e-12
<ndarray shape=() ctx=cpu(0)>
382 7.674713109088538e-12
<ndarray shape=() ctx=cpu(0)>
383 9.42301057942443e-12
<ndarray shape=() ctx=cp

551 1.021566892764423e-11
<ndarray shape=() ctx=cpu(0)>
552 8.929271130684495e-12
<ndarray shape=() ctx=cpu(0)>
553 8.579087215480376e-12
<ndarray shape=() ctx=cpu(0)>
554 8.796894273812488e-12
<ndarray shape=() ctx=cpu(0)>
555 9.158915067878672e-12
<ndarray shape=() ctx=cpu(0)>
556 9.293125693483244e-12
<ndarray shape=() ctx=cpu(0)>
557 9.051807877059241e-12
<ndarray shape=() ctx=cpu(0)>
558 1.0742645261443669e-11
<ndarray shape=() ctx=cpu(0)>
559 9.365230772416695e-12
<ndarray shape=() ctx=cpu(0)>
560 9.188709038446263e-12
<ndarray shape=() ctx=cpu(0)>
561 9.84750138488641e-12
<ndarray shape=() ctx=cpu(0)>
562 9.21088962277405e-12
<ndarray shape=() ctx=cpu(0)>
563 9.758748161365878e-12
<ndarray shape=() ctx=cpu(0)>
564 9.17139743666852e-12
<ndarray shape=() ctx=cpu(0)>
565 1.001175054318737e-11
<ndarray shape=() ctx=cpu(0)>
566 1.0237312384663239e-11
<ndarray shape=() ctx=cpu(0)>
567 1.0826731252109667e-11
<ndarray shape=() ctx=cpu(0)>
568 9.566852603219928e-12
<ndarray shape=() ctx=

751 8.907101689922163e-12
<ndarray shape=() ctx=cpu(0)>
752 9.657987327634301e-12
<ndarray shape=() ctx=cpu(0)>
753 8.745092977812405e-12
<ndarray shape=() ctx=cpu(0)>
754 7.91655421230296e-12
<ndarray shape=() ctx=cpu(0)>
755 9.770042932365436e-12
<ndarray shape=() ctx=cpu(0)>
756 1.0214581001914908e-11
<ndarray shape=() ctx=cpu(0)>
757 9.328536017651559e-12
<ndarray shape=() ctx=cpu(0)>
758 9.230322211992376e-12
<ndarray shape=() ctx=cpu(0)>
759 1.0172260657501792e-11
<ndarray shape=() ctx=cpu(0)>
760 8.833514350764862e-12
<ndarray shape=() ctx=cpu(0)>
761 8.78961613304881e-12
<ndarray shape=() ctx=cpu(0)>
762 1.005221978857382e-11
<ndarray shape=() ctx=cpu(0)>
763 1.024191896995875e-11
<ndarray shape=() ctx=cpu(0)>
764 1.0096850442455627e-11
<ndarray shape=() ctx=cpu(0)>
765 9.062041834160053e-12
<ndarray shape=() ctx=cpu(0)>
766 9.933034324666754e-12
<ndarray shape=() ctx=cpu(0)>
767 9.818911878229156e-12
<ndarray shape=() ctx=cpu(0)>
768 1.0040083049884033e-11
<ndarray shape=() ct

944 1.1502381786979024e-11
<ndarray shape=() ctx=cpu(0)>
945 1.0467569519501119e-11
<ndarray shape=() ctx=cpu(0)>
946 1.0000241208577877e-11
<ndarray shape=() ctx=cpu(0)>
947 1.0243532791339967e-11
<ndarray shape=() ctx=cpu(0)>
948 9.47169758599921e-12
<ndarray shape=() ctx=cpu(0)>
949 9.274851286295016e-12
<ndarray shape=() ctx=cpu(0)>
950 1.0230964191207956e-11
<ndarray shape=() ctx=cpu(0)>
951 9.669459423287301e-12
<ndarray shape=() ctx=cpu(0)>
952 9.072494396912024e-12
<ndarray shape=() ctx=cpu(0)>
953 9.823690787295588e-12
<ndarray shape=() ctx=cpu(0)>
954 8.963904230972937e-12
<ndarray shape=() ctx=cpu(0)>
955 8.905733277702688e-12
<ndarray shape=() ctx=cpu(0)>
956 8.144684918360602e-12
<ndarray shape=() ctx=cpu(0)>
957 9.048049598648538e-12
<ndarray shape=() ctx=cpu(0)>
958 8.417878431121609e-12
<ndarray shape=() ctx=cpu(0)>
959 8.787427128862562e-12
<ndarray shape=() ctx=cpu(0)>
960 8.872517416000933e-12
<ndarray shape=() ctx=cpu(0)>
961 9.407280621940304e-12
<ndarray shape=() 